# DRY/WET alapelvek

### **FOGALMAK:**
* **DRY** = **D**o not **R**epeat **Y**ourself
* **WET** = **W**rite **E**very **T**ime

A DRY és WET megközelítések nem zárják ki egymást feltétlenül. Például nem érdemes függvényt írni egy olyan feladatra, amit csak egyszer végzel el. Viszont jelentősen megnövelheti az olvashatóságát a kódodnak, amikor az adott feladat hosszú, illetve segít abban, hogy könnyedén tudd frissíteni a megoldásodat, anélkül hogy elvesznél az erdőben.

### Hogyan írjunk DRY kódot? -3 bevett stratégia van erre:
1. Írj függvényeket, amiket meg tudsz hívni. Így nem kell minden alkalmazáskor ugyanazt leírni
2. Írj loopokat, hogy a repetitív feladatokat iteratívan oldd meg
3. Írj feltételeket, hogy a különböző kimenetekre kontrollálni tudj (if-elif-else)

### Egy bevezető szemléltetése a WET kódnak:

In [1]:
print(1)
print(2)
print(3)
print(4)
print(5)

1
2
3
4
5


### Amit könnyedén lehet orvosolni egy for loop-al:

In [2]:
for i in range(1, 6):
    print(i)

1
2
3
4
5


### Itt egy másik WET kód példa:

In [3]:
x = y = 40

In [4]:
if x > y:
    z = x + 1
else:
    z = y + 2
print(z)

42


### Amit megint csak könnyen DRY-á lehet alakítani:
Így már nem kell minden eshetőségkor külön értéket adni z-nek

In [5]:
z = x + 1 if x > y else y + 2
print(z)

42


## Na de csapjunk bele valami szaftosabba!

## 1. Adat beolvasása

In [6]:
import pandas as pd
import numpy as np

df = pd.read_parquet(
    "https://borza-public-data.s3.eu-central-1.amazonaws.com/ha-data/ha-2019-05-18.parquet"
)

### Biztos jó a for loop?
* Ezt le ne futtassa senki (!)

In [ ]:
for index, row in df.iterrows():
    if row["horsepower"] <= 200:
        row["category"] = "gyenge"
    else:
        row["category"] = "bika"

### VAGY LEHET MÉG JAVÍTANI RAJTA?
* A NUMPY segítségünkre siet, igen!
* Látjuk, hogy sokkal DRY-abb
* De emellett az is előny, hogy lefut fényévekkel gyorsabban

In [7]:
df['category'] = np.where(df['horsepower'] <= 200, 'gyenge', 'bika')

## HIÁNYZÓ ÉRTÉKEK IMPUTÁLÁSA PÉLDA

### 1. HÁNY SZÁZALÉKA HIÁNYZIK AZ EGYES VÁLTOZÓKNAK?

In [8]:
df.isna().mean().sort_values()

parsed_price         0.000000
ÁLLÍTHATÓ KORMÁNY    0.000000
CD-RÁDIÓ             0.000000
KLÍMA                0.000000
ALUFELNI             0.000000
ESP                  0.000000
LÉGZSÁK              0.000000
FŰTHETŐ TÜKÖR        0.000000
ASR                  0.000000
KÖDLÁMPA             0.000000
ISOFIX               0.000000
BLUETOOTH            0.000000
FŰTHETŐ ÜLÉS         0.000000
CENTRÁLZÁR           0.000000
AUTOMATA             0.000000
BŐR                  0.000000
SZÍNEZETT ÜVEG       0.000000
GPS                  0.000000
FŰTHETŐ SZÉLVÉDŐ     0.000000
HIFI                 0.000000
cartype              0.000000
year                 0.000000
cc                   0.000000
horsepower           0.000000
brand                0.000000
PÓTKERÉK             0.000000
category             0.000000
fuel                 0.001543
kms                  0.007759
dtype: float64

### 2. PÓTOLJUK!

In [9]:
df["fuel"].fillna(0, inplace=True)
df["kms"].fillna(1, inplace=True)
df["power_per_price"] = df["horsepower"] / df["parsed_price"]

### 3. VAGY CSINÁLJUK ELEGÁNSABBAN:

írjuk meg előre a függvényekben a folyamatokat és utána a pipe operatorral hajtsuk végre egyben és átláthatóan a feladatokat!

In [10]:
def hianyzo_ertekek_imputalasa(df):
    df["fuel"].fillna(0, inplace=True)
    df["kms"].fillna(1, inplace=True)
    return df

def add_features(df):
    df["power_per_price"] = df["horsepower"] / df["parsed_price"]

és ha nem csak hiányzó értékeket imputálnánk, hanem mást is, akkor jelentősen átláthatóbb lesz a kódunk

### Használjuk a pipe operátort, ami lehetővé teszi, hogy láncszerűen hajtsunk végre az adatunkon függvényeket!

In [11]:
(
    df.pipe(hianyzo_ertekek_imputalasa)
    .pipe(add_features)
)

Nyilván, ez úgy igazán látványos, ha sok változó esetében imputálunk, akár eltérő módokon és nem egy új változót hozunk létre, hanem többet